In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2020-12-22 06:24:01,002 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-12-22 06:24:01,003 - settings - Setting database.user to celiib
INFO - 2020-12-22 06:24:01,004 - settings - Setting database.password to newceliipass
INFO - 2020-12-22 06:24:01,009 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-12-22 06:24:01,010 - settings - Setting enable_python_native_blobs to True
INFO - 2020-12-22 06:24:01,022 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-12-22 06:24:01,247 - settings - Setting enable_python_native_blobs to True


In [2]:
test_mode = False

In [3]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-12-22 06:24:02,276 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-12-22 06:24:02,278 - settings - Setting database.user to celiib
INFO - 2020-12-22 06:24:02,279 - settings - Setting database.password to newceliipass
INFO - 2020-12-22 06:24:02,286 - settings - Setting enable_python_native_blobs to True


Sleeping 157 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2020-12-22 06:24:02,503 - settings - Setting enable_python_native_blobs to True


# Getting the list of neurons to decompose for the mutli soma testing

In [ ]:
# import pandas as pd
# soma_soma_table = pd.read_csv("Minnie65 core proofreading - Soma-Soma.csv")
# no_header = soma_soma_table.iloc[1:]
# multi_soma_ids_str = no_header["Dendrites"].to_numpy()
# multi_soma_ids = multi_soma_ids_str[~np.isnan(multi_soma_ids_str.astype("float"))].astype("int")

# @schema
# class MultiSomaProofread(dj.Manual):
#     definition="""
#     segment_id : bigint unsigned  #segment id for those to be decimated
#     """
    
# dict_of_seg = [dict(segment_id=k) for k in multi_soma_ids]
# minnie.MultiSomaProofread.insert(dict_of_seg,skip_duplicates=True)
# MultiSomaProofread()

# Defining the Table

In [4]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [5]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 6377


In [6]:
#so that it will have the adapter defined
from datajoint_utils import *

In [9]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class DecompositionMultiSoma(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """

#     key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
#                             "decimation_version=" + str(decimation_version) &
#                        f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0" & "segment_id=864691136309663834").proj())
#     key_source = (minnie.Decimation() & "n_faces>500000").proj(decimation_version='version') & (minnie.BaylorSegmentCentroid() & "multiplicity=1").proj()
    key_source = (minnie.Decimation().proj(decimation_version='version')  & 
              dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
              & minnie.MultiSomaProofread()).proj()

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n----- Working on {segment_id}-------")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        somas = du.get_soma_mesh_list(segment_id) 
        print(f"somas = {somas}")
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")



        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [10]:
(minnie.schema.jobs & "table_name='__decomposition_multi_soma'")#.delete() #& "status='error'"
#((schema.jobs & "table_name = '__decomposition'") & "timestamp>'2020-11-16 00:26:00'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [ ]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 900))
print('Populate Started')
if not test_mode:
    DecompositionMultiSoma.populate(reserve_jobs=True, suppress_errors=True)
else:
    DecompositionMultiSoma.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")

INFO - 2020-12-22 06:26:07,219 - autopopulate - Found 171 keys to populate
INFO - 2020-12-22 06:26:07,229 - connection - Transaction started
INFO - 2020-12-22 06:26:07,231 - autopopulate - Populating: {'segment_id': 864691134988385914, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


Populate Started


----- Working on 864691134988385914-------
somas = [[<trimesh.Trimesh(vertices.shape=(24383, 3), faces.shape=(45990, 3))>], array([154.0564]), array([0.73])]

--- Beginning preprocessing of 864691134988385914---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(24383, 3), faces.shape=(45990, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(24383, 3), faces.shape=(45990, 3))>]
soma_mesh_list_centers = [array([1387335.35122831,  503651.28804905,  847195.67512611])]
xvfb-run -n 6377 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Soma_Splitting/temp/neuron_5788.off -o /notebooks/Auto_Proofreading/Soma_Splitting/temp/neuron_5788_remove_interior.off -s /notebooks/Auto_Proofreading/Soma_Splitting/temp/remove_interior_383187.mls
removed temporary input file: 

  0%|          | 0/1 [00:00<?, ?it/s]

Total time for mesh_pieces_connectivity= 4.637742519378662
# of insignificant_limbs = 3 with trimesh : [<trimesh.Trimesh(vertices.shape=(941, 3), faces.shape=(1847, 3))>, <trimesh.Trimesh(vertices.shape=(440, 3), faces.shape=(848, 3))>, <trimesh.Trimesh(vertices.shape=(362, 3), faces.shape=(666, 3))>]
# of not_processed_soma_containing_meshes = 0 with trimesh : []


----- Working on Proper Limb # 0 ---------
meshparty_segment_size = 100
root_curr = [1389704.   494435.4  849668.6]
Time for preparing soma vertices and root: 0.0002429485321044922
cc_vertex_thresh = 10


100%|██████████| 1/1 [00:01<00:00,  1.34s/it]

...

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.31s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 2.3855438232421875
connecting at the root
branches_touching_root = [28]
length of Graph = 10385
max(kept_branches_idx) = 28, len(kept_branches_idx) = 29
empty_indices % = 0.0
 conflict_indices % = 0.005629011683236479



AFTER face_lookup_resolved_test



Decomposing first pass: 11.763628959655762
Attempting to use MeshAfterParty Skeletonization and Mesh Correspondence
Another print
Found len(mesh_large_idx) MAP candidates: [13088, 4227, 60063, 2912, 11045, 1253, 455, 2905, 6161, 37630, 16465, 4493]
mesh_large_connectivity: 0.31830811500549316
Finding MAP candidates connected components: 0.00019621849060058594
len(filtered_pieces) = 1
skeleton_connectivity_MP : 0.2239360809326172
Grouping MP Sublimbs by Graph: 0.04534649848937988
Divinding into MP and MAP pieces: 9.5367431640625e-07
--- Working on MAP piece 0---
MAP Filtering Soma Pieces: 0.010866641998291016
inside skeletonize_connected_branch and use_surface_after_CGAL=False, surface_reconstruction_size=1000
The mesh passed has 12 pieces so just taking the largest one <trimesh.Trimesh(vertices.shape=(80694, 3), faces.shape=(160685, 3))>
     Starting Screened Poisson
